##**2. Word2Vec**
1. 주어진 단어들을 word2vec 모델에 들어갈 수 있는 형태로 만듭니다.
2. CBOW, Skip-gram 모델을 각각 구현합니다.
3. 모델을 실제로 학습해보고 결과를 확인합니다.

### **필요 패키지 import**

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 8.7MB/s 
     |████████████████████████████████| 460kB 51.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
from tqdm import tqdm
from konlpy.tag import Okt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

import torch
import copy
import numpy as np

### **데이터 전처리**



데이터를 확인하고 Word2Vec 형식에 맞게 전처리합니다.  
학습 데이터는 1번 실습과 동일하고, 테스트를 위한 단어를 아래와 같이 가정해봅시다.

In [ ]:
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."       
]

test_words = ["음식", "맛", "서비스", "위생", "가격"]

Tokenization과 vocab을 만드는 과정은 이전 실습과 유사합니다.

In [ ]:
tokenizer = Okt()

In [ ]:
def make_tokenized(data):
  tokenized = []
  for sent in tqdm(data):
    tokens = tokenizer.morphs(sent, stem=True)
    tokenized.append(tokens)

  return tokenized

In [ ]:
train_tokenized = make_tokenized(train_data)

100%|██████████| 10/10 [00:05<00:00,  1.75it/s]


In [ ]:
word_count = defaultdict(int)

for tokens in tqdm(train_tokenized):
  for token in tokens:
    word_count[token] += 1

100%|██████████| 10/10 [00:00<00:00, 2203.24it/s]


In [ ]:
word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
print(list(word_count))
len(word_count)

[('.', 14), ('도', 7), ('이다', 4), ('좋다', 4), ('별로', 3), ('다', 3), ('이', 3), ('너무', 3), ('음식', 3), ('서비스', 3), ('하다', 2), ('방문', 2), ('위생', 2), ('좀', 2), ('더', 2), ('에', 2), ('조금', 2), ('정말', 1), ('맛있다', 1), ('추천', 1), ('기대하다', 1), ('것', 1), ('보단', 1), ('가격', 1), ('비싸다', 1), ('다시', 1), ('가다', 1), ('싶다', 1), ('생각', 1), ('안', 1), ('드네', 1), ('요', 1), ('완전', 1), ('최고', 1), ('!', 1), ('재', 1), ('의사', 1), ('있다', 1), ('만족스럽다', 1), ('상태', 1), ('가', 1), ('개선', 1), ('되다', 1), ('기르다', 1), ('바라다', 1), ('맛', 1), ('직원', 1), ('분들', 1), ('친절하다', 1), ('기념일', 1), ('분위기', 1), ('전반', 1), ('적', 1), ('으로', 1), ('짜다', 1), ('저', 1), ('는', 1), ('신경', 1), ('써다', 1), ('불쾌하다', 1)]


60

In [ ]:
w2i = {}
for pair in tqdm(word_count):
  if pair[0] not in w2i:
    w2i[pair[0]] = len(w2i)

100%|██████████| 60/60 [00:00<00:00, 102592.03it/s]


In [ ]:
print(train_tokenized)
print(w2i)

[['정말', '맛있다', '.', '추천', '하다', '.'], ['기대하다', '것', '보단', '별로', '이다', '.'], ['다', '좋다', '가격', '이', '너무', '비싸다', '다시', '가다', '싶다', '생각', '이', '안', '드네', '요', '.'], ['완전', '최고', '이다', '!', '재', '방문', '의사', '있다', '.'], ['음식', '도', '서비스', '도', '다', '만족스럽다', '.'], ['위생', '상태', '가', '좀', '별로', '이다', '.', '좀', '더', '개선', '되다', '기르다', '바라다', '.'], ['맛', '도', '좋다', '직원', '분들', '서비스', '도', '너무', '친절하다', '.'], ['기념일', '에', '방문', '하다', '음식', '도', '분위기', '도', '서비스', '도', '다', '좋다', '.'], ['전반', '적', '으로', '음식', '이', '너무', '짜다', '.', '저', '는', '별로', '이다', '.'], ['위생', '에', '조금', '더', '신경', '써다', '좋다', '.', '조금', '불쾌하다', '.']]
{'.': 0, '도': 1, '이다': 2, '좋다': 3, '별로': 4, '다': 5, '이': 6, '너무': 7, '음식': 8, '서비스': 9, '하다': 10, '방문': 11, '위생': 12, '좀': 13, '더': 14, '에': 15, '조금': 16, '정말': 17, '맛있다': 18, '추천': 19, '기대하다': 20, '것': 21, '보단': 22, '가격': 23, '비싸다': 24, '다시': 25, '가다': 26, '싶다': 27, '생각': 28, '안': 29, '드네': 30, '요': 31, '완전': 32, '최고': 33, '!': 34, '재': 35, '의사': 36, '있다': 37, '만족스럽다': 38, '상태

실제 모델에 들어가기 위한 input을 만들기 위해 `Dataset` 클래스를 정의합니다.

In [ ]:
class CBOWDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.x.append(token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.y.append(id)

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수, 2 * window_size)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [ ]:
class SkipGramDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.y += (token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.x += [id] * 2 * window_size

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

각 모델에 맞는 `Dataset` 객체를 생성합니다.

In [ ]:
cbow_set = CBOWDataset(train_tokenized)
skipgram_set = SkipGramDataset(train_tokenized)
print(list(skipgram_set))

100%|██████████| 10/10 [00:00<00:00, 17795.10it/s]

[(tensor(0), tensor(17)), (tensor(0), tensor(18)), (tensor(0), tensor(19)), (tensor(0), tensor(10)), (tensor(19), tensor(18)), (tensor(19), tensor(0)), (tensor(19), tensor(10)), (tensor(19), tensor(0)), (tensor(22), tensor(20)), (tensor(22), tensor(21)), (tensor(22), tensor(4)), (tensor(22), tensor(2)), (tensor(4), tensor(21)), (tensor(4), tensor(22)), (tensor(4), tensor(2)), (tensor(4), tensor(0)), (tensor(23), tensor(5)), (tensor(23), tensor(3)), (tensor(23), tensor(6)), (tensor(23), tensor(7)), (tensor(6), tensor(3)), (tensor(6), tensor(23)), (tensor(6), tensor(7)), (tensor(6), tensor(24)), (tensor(7), tensor(23)), (tensor(7), tensor(6)), (tensor(7), tensor(24)), (tensor(7), tensor(25)), (tensor(24), tensor(6)), (tensor(24), tensor(7)), (tensor(24), tensor(25)), (tensor(24), tensor(26)), (tensor(25), tensor(7)), (tensor(25), tensor(24)), (tensor(25), tensor(26)), (tensor(25), tensor(27)), (tensor(26), tensor(24)), (tensor(26), tensor(25)), (tensor(26), tensor(27)), (tensor(26), tens

### **모델 Class 구현**

차례대로 두 가지 Word2Vec 모델을 구현합니다.  


*   `self.embedding`: `vocab_size` 크기의 one-hot vector를 특정 크기의 `dim` 차원으로 embedding 시키는 layer.
*   `self.linear`: 변환된 embedding vector를 다시 원래 `vocab_size`로 바꾸는 layer.


In [ ]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, dim):
    super(CBOW, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x):  # x: (B, 2W)
    embeddings = self.embedding(x)  # (B, 2W, d_w)
    embeddings = torch.sum(embeddings, dim=1)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

In [ ]:
class SkipGram(nn.Module):
  def __init__(self, vocab_size, dim):
    super(SkipGram, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x): # x: (B)
    embeddings = self.embedding(x)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

두 가지 모델을 생성합니다.

In [ ]:
cbow = CBOW(vocab_size=len(w2i), dim=256)
skipgram = SkipGram(vocab_size=len(w2i), dim=256)

In [ ]:
embedding = nn.Embedding(len(w2i), 256, sparse=True)
embedding(torch.LongTensor([1,2,3]))

tensor([[ 8.6567e-01, -1.4645e-01, -1.7260e-01,  1.1016e+00, -2.2692e-01,
         -2.6018e-01, -9.1777e-01,  9.7721e-01,  2.3111e+00, -6.3375e-01,
          1.9866e+00, -5.4849e-01,  2.1429e+00, -1.7132e-01, -3.3334e-01,
          1.0506e+00, -6.7445e-02,  7.2471e-01,  8.0551e-01,  4.9456e-01,
         -8.9204e-01,  1.1289e-02,  1.5093e-02,  4.5914e-01,  8.0744e-01,
          2.0440e+00,  6.6584e-01, -1.2712e+00,  2.2689e+00, -4.3072e-01,
          8.6404e-01,  1.2937e-01, -8.6559e-01, -1.9108e+00,  3.1215e-01,
         -1.4401e+00, -2.0910e-01,  2.6681e-01,  1.6035e+00, -8.1611e-02,
          1.8606e-01, -7.1135e-01,  6.6594e-01,  1.2554e+00,  2.2595e-01,
          1.2697e-01, -1.7966e-01, -4.7954e-01, -1.3816e-01, -5.9504e-01,
         -3.5332e-01,  1.0411e+00, -2.8073e-01, -8.2438e-01,  8.1640e-01,
          3.5476e-01,  2.6893e-01, -1.0454e-01, -1.4622e+00,  6.8922e-01,
         -1.5252e-02, -1.2040e+00,  1.0762e-01,  5.1284e-02, -2.2886e+00,
          5.7513e-01,  4.9375e-01,  3.

In [ ]:
embedding = nn.Embedding(len(w2i), 256, sparse=False)
embedding(torch.LongTensor([1,2,3]))

tensor([[-2.7733e-01, -4.8784e-01, -1.3065e+00,  1.5167e-01, -8.3360e-01,
          9.9477e-01,  1.0052e-01, -2.9109e-01, -1.1010e-01, -2.3719e+00,
          1.9117e-01, -1.0184e+00,  4.1440e-01,  6.4197e-01, -5.4189e-01,
          7.2439e-01,  1.8347e+00, -4.1957e-01,  2.1437e+00, -7.4768e-01,
          1.7846e+00, -1.2650e+00,  6.4252e-01,  1.0835e+00, -1.9495e+00,
          3.6300e-01, -1.1750e+00, -6.1529e-01, -2.8906e-01, -5.4641e-01,
          5.3123e-01,  1.3701e-01,  4.8498e-01, -1.7590e+00,  9.2524e-01,
          5.0298e-02, -9.2352e-01,  8.7167e-01,  7.1414e-01,  8.0263e-01,
          1.7196e-01,  1.4646e+00, -9.5186e-01, -6.2383e-01, -1.0177e+00,
         -8.3944e-01,  9.3505e-01, -4.7513e-01, -3.0175e-01, -3.9284e-01,
         -3.1819e-01,  8.5353e-01,  3.6175e-01,  7.0879e-01,  1.0861e-01,
          1.1440e+00, -3.1504e-01, -2.1141e-01, -1.3175e+00,  2.7365e-01,
          5.7825e-01,  4.9810e-01,  3.7276e-01, -3.2029e-01, -9.2078e-01,
          5.5937e-01, -6.2729e-02,  6.

### **모델 학습**

다음과 같이 hyperparamter를 세팅하고 `DataLoader` 객체를 만듭니다.

In [ ]:
batch_size=4
learning_rate = 5e-4
num_epochs = 5
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

cbow_loader = DataLoader(cbow_set, batch_size=batch_size)
skipgram_loader = DataLoader(skipgram_set, batch_size=batch_size)

첫번째로 CBOW 모델 학습입니다.

In [ ]:
cbow.train()
cbow = cbow.to(device)
optim = torch.optim.SGD(cbow.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(cbow_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = cbow(x)  # (B, V)
 
    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  0%|          | 0/16 [00:00<?, ?it/s]

##################################################
Epoch: 1
Train loss: 4.754945755004883
Train loss: 4.290592193603516
Train loss: 4.213982105255127
Train loss: 4.430968284606934
Train loss: 5.957576751708984
Train loss: 5.598806381225586
Train loss: 4.238408088684082
Train loss: 4.729179382324219
Train loss: 4.197514533996582
Train loss: 4.699819087982178
Train loss: 4.429537773132324
Train loss: 5.448338985443115
Train loss: 4.828976631164551
Train loss: 4.073480606079102
Train loss: 5.221699237823486
Train loss: 4.474963665008545
##################################################
Epoch: 2


100%|██████████| 16/16 [00:00<00:00, 660.48it/s]

Train loss: 4.576406478881836
Train loss: 4.166545867919922
Train loss: 4.107646465301514
Train loss: 4.317821025848389
Train loss: 5.800015449523926
Train loss: 5.3189697265625
Train loss: 4.0627875328063965
Train loss: 4.598029136657715
Train loss: 4.089056015014648
Train loss: 4.522393703460693
Train loss: 4.2632598876953125
Train loss: 5.084076881408691
Train loss: 4.690461158752441
Train loss: 3.956787109375
Train loss: 5.054161071777344
Train loss: 4.328393936157227
##################################################
Epoch: 3
Train loss: 4.402919292449951
Train loss: 4.044163703918457
Train loss: 4.002156734466553
Train loss: 4.206272125244141
Train loss: 5.644565582275391
Train loss: 5.0439982414245605
Train loss: 3.8924477100372314
Train loss: 4.470180511474609
Train loss: 3.9837160110473633
Train loss: 4.347694396972656
Train loss: 4.10220193862915
Train loss: 4.729652404785156
Train loss: 4.5533366203308105
Train loss: 3.8451924324035645
Train loss: 4.8904008865356445
Train lo

다음으로 Skip-gram 모델 학습입니다.

In [ ]:
skipgram.train()
skipgram = skipgram.to(device)
optim = torch.optim.SGD(skipgram.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(skipgram_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = skipgram(x)  # (B, V)

    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  0%|          | 0/64 [00:00<?, ?it/s]

##################################################
Epoch: 1
Train loss: 4.269641876220703
Train loss: 4.14031982421875
Train loss: 4.632961273193359
Train loss: 4.0716986656188965
Train loss: 3.882753372192383
Train loss: 4.412576675415039
Train loss: 4.367706775665283
Train loss: 3.9476544857025146
Train loss: 4.0739030838012695
Train loss: 4.23362922668457
Train loss: 4.2539286613464355
Train loss: 4.201377868652344
Train loss: 4.325809001922607
Train loss: 4.371469974517822
Train loss: 4.195186138153076
Train loss: 4.898489952087402
Train loss: 4.280550479888916
Train loss: 4.210625648498535
Train loss: 4.420271396636963
Train loss: 4.4148712158203125
Train loss: 4.293399333953857
Train loss: 4.218382835388184
Train loss: 4.238741397857666
Train loss: 4.206324577331543
Train loss: 4.291888236999512
Train loss: 4.217133522033691
Train loss: 4.040428161621094
Train loss: 4.43715238571167
Train loss: 4.300670623779297
Train loss: 3.71821928024292
Train loss: 4.600168228149414
Train los

  0%|          | 0/64 [00:00<?, ?it/s]

Train loss: 4.363997459411621
Train loss: 4.357032775878906
Train loss: 4.123373985290527
Train loss: 4.049276351928711
Train loss: 4.1624603271484375
Train loss: 4.14983606338501
Train loss: 4.220363616943359
Train loss: 4.1130571365356445
Train loss: 3.9556093215942383
Train loss: 4.37733793258667
Train loss: 4.216354846954346
Train loss: 3.6578257083892822
Train loss: 4.532642841339111
Train loss: 4.042409896850586
Train loss: 4.482714653015137
Train loss: 4.02559757232666
Train loss: 4.1271467208862305
Train loss: 4.082010746002197
Train loss: 3.4450018405914307
Train loss: 3.7285165786743164
Train loss: 4.125820159912109
Train loss: 4.243056297302246
Train loss: 4.201876163482666
Train loss: 3.691709041595459
Train loss: 4.517106056213379
Train loss: 4.040648460388184
Train loss: 4.00918436050415
Train loss: 3.901686668395996
Train loss: 4.099496841430664
Train loss: 3.8340561389923096
Train loss: 3.4962081909179688
Train loss: 4.3055596351623535
Train loss: 4.208200931549072
Trai

100%|██████████| 64/64 [00:00<00:00, 764.87it/s]

Train loss: 3.361055850982666
Train loss: 3.6488373279571533
Train loss: 4.037426471710205
Train loss: 4.188492774963379
Train loss: 4.1436872482299805
Train loss: 3.6483726501464844
Train loss: 4.399072647094727
Train loss: 3.937584400177002
Train loss: 3.7891464233398438
Train loss: 3.7381653785705566
Train loss: 3.9301445484161377
Train loss: 3.761371612548828
Train loss: 3.4451000690460205
Train loss: 4.255953311920166
Train loss: 4.134788990020752
Train loss: 4.1282639503479
Train loss: 3.9124855995178223
Train loss: 3.955836057662964
Train loss: 3.7561070919036865
Train loss: 4.0261311531066895
Train loss: 3.8622121810913086
Train loss: 4.063629150390625
Train loss: 4.011898040771484
Train loss: 4.2000555992126465
Train loss: 3.899550676345825
Train loss: 4.336552619934082
Train loss: 4.261462211608887
Train loss: 4.271452903747559
Finished.


### **테스트**

학습된 각 모델을 이용하여 test 단어들의 word embedding을 확인합니다.

In [ ]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = cbow.embedding(input_id)

  print(f"Word: {word}")
  print(emb.squeeze(0))

Word: 음식
tensor([-8.7940e-01, -7.3004e-01,  1.0023e+00, -3.8496e-01,  4.3666e-03,
        -7.1984e-02, -5.5287e-02,  8.9894e-01, -1.0305e+00, -1.1356e+00,
         5.6411e-01,  1.3044e-01, -3.7066e-01,  7.2932e-01, -3.1032e-01,
         1.0960e+00, -1.7473e-01,  1.0919e+00,  1.8430e-02, -6.2172e-01,
        -7.6097e-01, -9.7185e-01, -8.5380e-01, -9.0724e-01,  4.8650e-01,
         5.7786e-01,  1.1306e+00, -5.4112e-01, -1.3561e+00,  4.0185e-01,
         1.3755e+00, -1.2558e+00,  9.3634e-01,  3.9461e-01, -6.8009e-02,
         9.9459e-01,  3.5928e-01, -2.6833e+00,  4.2685e-01, -3.6337e-01,
        -1.0154e+00, -2.0316e-01, -8.4114e-01,  8.7546e-01,  2.8669e-01,
        -3.2736e-01, -4.3495e-01, -1.5923e+00,  1.3950e+00,  5.3030e-02,
         2.0568e+00,  5.0999e-01,  3.8126e-01, -1.7067e-01, -4.0084e-01,
         1.1872e+00,  7.2028e-01,  6.4824e-01,  4.5702e-01, -1.0367e+00,
        -4.7624e-01,  1.6377e+00,  5.3392e-01,  1.4296e-01,  3.6640e-01,
        -6.4731e-01,  5.1232e-01,  1.1209e

In [ ]:
for word in test_words:

  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = skipgram.embedding(input_id)

  print(f"Word: {word}")
  print(emb.squeeze(0))

Word: 음식
tensor([-0.4028,  0.1612,  0.3360, -1.0856,  2.3051,  0.7446, -0.4309, -0.8757,
         0.9802, -0.8527, -0.8463, -0.6561, -0.1993, -0.0058,  2.3992,  0.4526,
        -0.9500, -0.5230, -0.7011, -1.0525, -1.9706,  1.6420,  1.7454, -1.2930,
         0.2302,  0.7838, -1.4744,  0.4100, -0.4302, -1.3660,  0.1505,  0.5202,
         1.3057, -0.3662,  1.2751, -0.7210, -1.2883, -0.7870,  0.2205, -0.7886,
         0.4784,  1.8578,  1.3035,  0.0659, -1.9863, -1.0185,  0.1364, -0.2422,
        -0.8939, -0.5392, -0.1992, -1.8788,  0.1251,  0.3758,  1.0502, -1.0988,
        -1.0099,  0.2536,  0.3273, -1.2719, -0.0048,  0.0693, -2.5666,  0.7108,
        -0.4248,  0.0345, -1.4300, -0.0691,  1.0977, -0.2731, -0.9515, -0.8822,
         0.1936, -0.7494,  2.3647,  0.7400, -0.3946,  0.4903, -1.1521,  0.1030,
         0.0621,  0.2775,  0.3751,  0.9269, -1.8535, -1.9377, -0.1461, -1.4735,
        -0.3034, -1.6798, -0.6237, -0.4715, -1.4069,  0.1719, -1.6662,  0.0228,
        -1.8699, -0.8817,  0.65